# Point Process

In [1]:
import pandas as pd
import numpy as np
import theano
import pymc3 as pm
from scipy.stats import ks_2samp
import seaborn as sns
import matplotlib.pyplot as plt
import glob

sns.set_context('notebook')
plt.style.use('seaborn-darkgrid')
print('Running on PyMC3 v{}'.format(pm.__version__))
#theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"
import warnings

Running on PyMC3 v3.8


## Dirichlet Mixture Model

In [3]:
def Bayesian_mixture_model(ISI_data):
    
    with pm.Model() as model:
        
        BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0,upper=0.2)
        sigma = pm.Uniform('sigma',lower=0.0001,upper=0.5)
        obs2 = pm.Bound(pm.Normal, lower=0.0).dist(mu=mu2,sigma=sigma)

        mu3 = pm.Uniform('mu3',lower=0.1,upper=0.6)
        sigma3 = pm.Uniform('sigma3',lower=0.0001,upper=0.5)
        obs3 = pm.Bound(pm.Normal, lower=0.0).dist(mu=mu3,sigma=sigma3)


        w = pm.Dirichlet('w', a=np.array([1., .4, .4]))

        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3], observed=ISI_data)


        #step = pm.Metropolis(target_accept=0.9)
        #trace = pm.sample(step=step,draws=500,tune=100,progressbar=False,random_seed=10,cores=6)
     

    map_estimate = pm.find_MAP(model=model)
    del map_estimate['w_stickbreaking__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma_interval__']
    del map_estimate['mu3_interval__']
    del map_estimate['sigma3_interval__']
    
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    map_estimate['w3'] = map_estimate['w'][2]
    
    del map_estimate['w']
    
    map_estimate['n_spikes'] = ISI_data.shape[0]
    
    
    #with model:
    #    ppc_trace = pm.sample_posterior_predictive(trace, 50,model=model,progressbar=False,random_seed=10)
    
    #lista_samples=[]
    #for i in list(ppc_trace['like']):
    #    lista_samples.extend(i)
    
    
    #print('P_value: ',ks_2samp(lista_samples,ISI_data,mode = 'asymp').pvalue)
    
    return map_estimate

## Inverse Gaussian

In [4]:
def Inverse_Gaussian_model(ISI_data):    
    
    with pm.Model() as model:
        BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald('like',mu=mu1,lam=lam1,observed=ISI_healthy)


        step = pm.NUTS(target_accept=0.9)
        trace = pm.sample(step=step,draws=1000,tune=1000,cores=6)

        map_estimate = pm.find_MAP(model=model)
        
    
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    
    
    
    
    map_estimate['n_spikes'] = ISI_data.shape[0]
    
    
    with model:
        ppc_trace = pm.sample_posterior_predictive(trace, 200,model=model)
    
    lista_samples=[]
    for i in list(ppc_trace['like']):
        lista_samples.extend(i)
    
    
    print('P_value: ',ks_2samp(lista_samples,ISI_data,mode = 'asymp').pvalue)
    
    return map_estimate

## Import all neural net

In [5]:
list_dir=glob.glob("Data after SS/*.txt")

In [6]:
for net in list_dir:
    if 'healthy' in net:
        print(net,'ok')

Data after SS/2018-10-12T15-33-52MIP nr 5, baseline,control, H9N healthy.h5.txt ok
Data after SS/2019-01-23T11-19-05PoP1 healthy cortical .h5.txt ok
Data after SS/2019-01-23T11-30-29MEA2 healthy cortical .h5.txt ok
Data after SS/2019-01-23T11-41-41Pop1 healthy cortical .h5.txt ok
Data after SS/2019-01-23T11-05-52MIP3 healthy cortical .h5.txt ok
Data after SS/2018-10-12T14-03-45MIP nr 2, baseline,  control, H9N healthy.h5.txt ok


In [9]:
list_neurons = np.genfromtxt(net, delimiter=',')

In [12]:
neuron=list_neurons[0]
neuron=neuron[neuron>100*10000]
neuron=neuron[neuron<300*10000]
neuron

array([1000226., 1001448., 1006196., ..., 2997854., 2998410., 2999155.])

## Run the model on the interval 100-300s

In [17]:
dataframe = pd.DataFrame()
counter_net=1
counter=0
target = 0
for net in list_dir:
    print(counter_net,') ',net)
    counter_net+=1
    list_neurons = np.genfromtxt(net, delimiter=',')
    counter=0
    print('Original number of neurons: ',len(list_neurons))
    for neuron in list_neurons:
        neuron=neuron[neuron>100*10000]
        neuron=neuron[neuron<300*10000]
        if neuron.shape[0]>1000:
            
            counter+=1
        else:
            print('Excluded neuron with n spikes = ',neuron.shape[0])
            continue
        
        ISI_healthy = np.diff(neuron)/10000
        
            
        map_estimate = Bayesian_mixture_model(ISI_healthy)
        
        if 'healthy' in net:
            map_estimate['Target']=0
            target = 0
        elif 'health' in net:
            map_estimate['Target']=0
            target = 0
        else:
            map_estimate['Target']=1
            target = 1
        df = pd.DataFrame.from_dict(map_estimate,orient='index')
        dataframe = pd.concat([dataframe,df],axis = 1)
    print('Final number of neurons: ',counter)
    print('Target = ',target)

1 )  Data after SS/2018-10-12T15-33-52MIP nr 5, baseline,control, H9N healthy.h5.txt
Original number of neurons:  94


logp = 621.48, ||grad|| = 0.028659: 100%|██████████| 54/54 [00:00<00:00, 944.03it/s]  
logp = 31,589, ||grad|| = 0.89338: 100%|██████████| 55/55 [00:00<00:00, 371.97it/s]  


Excluded neuron with n spikes =  113
Excluded neuron with n spikes =  488


logp = 9,920, ||grad|| = 0.11889: 100%|██████████| 41/41 [00:00<00:00, 581.37it/s]   


Excluded neuron with n spikes =  378
Excluded neuron with n spikes =  957


...

Excluded neuron with n spikes =  546


logp = 1,594.3, ||grad|| = 0.033688: 100%|██████████| 37/37 [00:00<00:00, 880.47it/s]  


Excluded neuron with n spikes =  553


logp = 6,822.6, ||grad|| = 0.05688: 100%|██████████| 54/54 [00:00<00:00, 744.51it/s]  
logp = 1,572.3, ||grad|| = 0.0084068: 100%|██████████| 58/58 [00:00<00:00, 950.71it/s]  

Final number of neurons:  37
Target =  1


In [18]:
final = dataframe.T
final.to_csv('Data after PP/Data')